# PyTorch中的调试和可视化

本文主要参考以下内容简单了解PyTorch中的调试和可视化以帮助检查反向传播过程。

- [PyTorch 101, Part 5: Understanding Hooks](https://blog.paperspace.com/pytorch-hooks-gradient-clipping-debugging/)

PyTorch 中的钩子的功能严重的缺乏记录。通常我们都没听说过他，然而它非常有用。

钩子重要的原因之一是它可以让我们在反向传播过程中做一些事情。钩子就像是许多英雄留在恶棍巢穴中以获取所有信息的设备之一。

你可以在 一个张量或者一个nn.Module上注册一个钩子。钩子实际上就是一个函数，在调用forward 或 backward 时执行。

这里的forward意思不是nn.Module的forward。这里的forward  函数是torch.Autograd.Function对象（张量的grad_fn）的函数。注意前面的文件记录过：每个张量都有一个创建该张量torch.Autograd.Function对象--grad_fn。例如，如果一个张量是由 tens = tens1 + tens2 创建的，那么它grad_fn就是AddBackward。

请注意，一个 nn.Module，比如 ann.Linear，有多个forward调用。它的输出是由两个运算创建的 (Y = W * X + B)，加法和乘法，因此会有两个forward调用。这可能会把事情搞砸，并可能导致多个输出。我们将在本文后面更详细地讨论这一点。

PyTorch 提供了两种类型的钩子。

1. 前向钩子 The Forward Hook
2. 后向钩子 The Backward Hook

前向钩子在前向传递期间执行，而后向钩子在backward被调用时执行。再次提醒，这些forward和backward都是一个Autograd.Function对象的函数。

## 张量钩子

钩子是一个函数，具有非常具体的signature。当我们说一个钩子被执行时，实际上我们是在谈论这个正在执行的函数。

对于张量，backward hook 的signature是，

```Python
hook(grad) -> Tensor or None
```

forward张量没有钩子。

grad是在调用backward后张量的grad属性中包含的值。不支持该函数修改它的参数。它必须返回None或用于代替grad作进一步梯度计算的张量。下面提供一个示例。

In [1]:
import torch 
a = torch.ones(5)
a.requires_grad = True

b = 2*a

# https://pytorch.org/docs/stable/autograd.html
b.retain_grad()   # Since b is non-leaf and its grad will be destroyed otherwise.

c = b.mean()

c.backward()

print(a.grad, b.grad)

tensor([0.4000, 0.4000, 0.4000, 0.4000, 0.4000]) tensor([0.2000, 0.2000, 0.2000, 0.2000, 0.2000])


C:\Users\11445\miniconda3\envs\hydrus\lib\site-packages\torch\autograd\__init__.py:132: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  allow_unreachable=True)  # allow_unreachable flag


In [2]:
# Redo the experiment but with a hook that multiplies b's grad by 2. 
a = torch.ones(5)

a.requires_grad = True

b = 2*a

b.retain_grad()

b.register_hook(lambda x: print(x))  

b.mean().backward() 


print(a.grad, b.grad)

tensor([0.2000, 0.2000, 0.2000, 0.2000, 0.2000])
tensor([0.4000, 0.4000, 0.4000, 0.4000, 0.4000]) tensor([0.2000, 0.2000, 0.2000, 0.2000, 0.2000])


如上所述，钩子有多种用途。

1. 可以打印梯度值进行调试。也可以记录它们。这对于梯度被释放的非叶变量特别有用，除非对它们调用retain_grad。执行后者会增加内存保留。Hooks 提供了更简洁的方式来聚合这些值。
2. 可以在向后传递期间修改梯度。这个非常重要。虽然仍然可以访问grad网络中张量的grad变量，但只能在整个向后传递完成后才能访问它。例如，我们将b的梯度乘以 2，现在后续的梯度计算，如 a（或任何依赖于b梯度的张量），使用 2 * grad(b) 而不是 grad(b)。相反，如果我们在backward后单独更新参数，我们不得不将b.grad以及a.grad乘2。

In [3]:
a = torch.ones(5)

a.requires_grad = True
b = 2*a

b.retain_grad()


b.mean().backward() 


print(a.grad, b.grad)

b.grad *= 2

print(a.grad, b.grad)       # a's gradient needs to updated manually

tensor([0.4000, 0.4000, 0.4000, 0.4000, 0.4000]) tensor([0.2000, 0.2000, 0.2000, 0.2000, 0.2000])
tensor([0.4000, 0.4000, 0.4000, 0.4000, 0.4000]) tensor([0.4000, 0.4000, 0.4000, 0.4000, 0.4000])


## nn.Module 对象的钩子

对于nn.Module对象，钩子函数的signature ，

对于后钩，

```Python
hook(module, grad_input, grad_output) -> Tensor or None
```

```Python
hook(module, input, output) -> None
```

为前钩。

在开始之前，明确一点，在nn.Module对象上使用钩子迫使我们打破抽象。Ann.Module应该是表示层的模块化对象。但是，hook受 forward和 backward的约束，其中nn.Module对象中可以有任意数量。这需要我们了解模块化对象的内部结构。

例如， nn.Linear在执行期间涉及两个forward调用。乘法和加法 (y = w * x + b)。这就是为什么input钩子函数可以是一个包含两个不同forward调用的输入和前向调用的输出的元组。

grad_input是nn.Module对象的输入对loss的梯度（dL/dx，dL/dw，dL/b）。grad_output是nn.Module对象的输出对梯度输出的梯度。由于在一个nn.Module对象内进行多次调用，这些可能非常不明确。

考虑以下代码。

In [4]:
import torch 
import torch.nn as nn

class myNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(3,10,2, stride = 2)
        self.relu = nn.ReLU()
        self.flatten = lambda x: x.view(-1)
        self.fc1 = nn.Linear(160,5)
   
  
    def forward(self, x):
        x = self.relu(self.conv(x))
        return self.fc1(self.flatten(x))

net = myNet()

def hook_fn(m, i, o):
    print(m)
    print("------------Input Grad------------")
    for grad in i:
        try:
            print(grad.shape)
        except AttributeError: 
            print ("None found for Gradient")
    
    print("------------Output Grad------------")
    for grad in o:  
        try:
            print(grad.shape)
        except AttributeError: 
            print ("None found for Gradient")
    print("\n")

net.conv.register_backward_hook(hook_fn)
net.fc1.register_backward_hook(hook_fn)
inp = torch.randn(1,3,8,8)
out = net(inp)

(1 - out.mean()).backward()

Linear(in_features=160, out_features=5, bias=True)
------------Input Grad------------
torch.Size([5])
torch.Size([5])
------------Output Grad------------
torch.Size([5])


Conv2d(3, 10, kernel_size=(2, 2), stride=(2, 2))
------------Input Grad------------
None found for Gradient
torch.Size([10, 3, 2, 2])
torch.Size([10])
------------Output Grad------------
torch.Size([1, 10, 4, 4])




在上面的代码中，使用的钩打印grad_input和grad_output的形状。但无法弄清楚grad_input应该代表什么。

## Hooks 的正确使用方法

所以，全力支持在 Tensor 上使用钩子。使用named_parameters函数，能成功地使用 PyTorch 完成所有的梯度修改/裁剪需求。named_parameters允许我们更好地控制要修改的梯度。这么说吧，想做两件事。

1. 在反向传播时将线性偏差的梯度变为零。
2. 确保没有小于 0的梯度进入 conv 层。

In [5]:
import torch 
import torch.nn as nn

class myNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(3,10,2, stride = 2)
        self.relu = nn.ReLU()
        self.flatten = lambda x: x.view(-1)
        self.fc1 = nn.Linear(160,5)
    def forward(self, x):
        x = self.relu(self.conv(x))
        x.register_hook(lambda grad : torch.clamp(grad, min = 0))     #No gradient shall be backpropagated 
                                                                  #conv outside less than 0
      
        # print whether there is any negative grad
        x.register_hook(lambda grad: print("Gradients less than zero:", bool((grad < 0).any())))  
        return self.fc1(self.flatten(x))
    
net = myNet()

for name, param in net.named_parameters():
  # if the param is from a linear and is a bias
  if "fc" in name and "bias" in name:
    param.register_hook(lambda grad: torch.zeros(grad.shape))


out = net(torch.randn(1,3,8,8)) 

(1 - out).mean().backward()

print("The biases are", net.fc1.bias.grad)     #bias grads are zero

Gradients less than zero: False
The biases are tensor([0., 0., 0., 0., 0.])


## 可视化激活的前向钩子

如果您注意到，Tensor没有前向钩子，而nn.Module有一个，在调用forward 时执行。尽管已经强调了将钩子附加到 PyTorch 的问题，但能看到很多人使用前向钩子通过将特征图保存到钩子函数外部的 python 变量来保存中间特征图。

In [6]:
visualisation = {}

inp = torch.randn(1,3,8,8)

def hook_fn(m, i, o):
    visualisation[m] = o 

net = myNet()

for name, layer in net._modules.items():
    layer.register_forward_hook(hook_fn)

out = net(inp) 

通常，nn.Module的output是最后一个forward的输出。但是，可以在不使用钩子的情况下安全地复制上述功能。只需简单地将nn.Module对象的forward函数的中间输出附加到列表中。但是，在 nn.Sequential中打印模块的中间激活可能有点问题。为了解决这个问题，我们需要注册一个到 Sequential 的子模块的钩子，而不是Sequential它本身。

In [7]:
import torch 
import torch.nn as nn

class myNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(3,10,2, stride = 2)
        self.relu = nn.ReLU()
        self.flatten = lambda x: x.view(-1)
        self.fc1 = nn.Linear(160,5)
        self.seq = nn.Sequential(nn.Linear(5,3), nn.Linear(3,2))
    def forward(self, x):
        x = self.relu(self.conv(x))
        x = self.fc1(self.flatten(x))
        x = self.seq(x)

net = myNet()
visualisation = {}

def hook_fn(m, i, o):
    visualisation[m] = o 

def get_all_layers(net):
    for name, layer in net._modules.items():
    #If it is a sequential, don't register a hook on it
    # but recursively register hook on all it's module children
        if isinstance(layer, nn.Sequential):
            get_all_layers(layer)
        else:
      # it's a non sequential. Register a hook
            layer.register_forward_hook(hook_fn)

get_all_layers(net)

  
out = net(torch.randn(1,3,8,8))

# Just to check whether we got all layers
visualisation.keys()      #output includes sequential layers

dict_keys([Conv2d(3, 10, kernel_size=(2, 2), stride=(2, 2)), ReLU(), Linear(in_features=160, out_features=5, bias=True), Linear(in_features=5, out_features=3, bias=True), Linear(in_features=3, out_features=2, bias=True)])

最后，可以将此张量转换为 numpy 数组并绘制激活图。